
# 🌼 RAG기법의 이해와 적용 - 4차시(24.12.03)

---

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
# pip install pdfplumber

In [3]:
from langchain_teddynote import logging
logging.langsmith("CLASS")

LangSmith 추적을 시작합니다.
[프로젝트명]
CLASS


In [4]:
from langchain_community.document_loaders import PDFPlumberLoader

In [5]:
loader = PDFPlumberLoader('./data/SPRi AI Brief_11월호_산업동향_F.pdf')

In [6]:
docs = loader.load()

In [7]:
docs = docs[3: -1]

In [8]:
len(docs)

21

In [9]:
docs

[Document(metadata={'source': './data/SPRi AI Brief_11월호_산업동향_F.pdf', 'file_path': './data/SPRi AI Brief_11월호_산업동향_F.pdf', 'page': 3, 'total_pages': 25, 'Author': 'dj', 'Creator': 'Hwp 2018 10.0.0.13947', 'Producer': 'Hancom PDF 1.3.0.547', 'CreationDate': "D:20241105150426+09'00'", 'ModDate': "D:20241105150426+09'00'", 'PDFVersion': '1.4'}, page_content='1. 정책/법제 2. 기업/산업 3. 기술/연구 4. 인력/교육\n미국 민권위원회, 연방정부의 얼굴인식 기술 사용에 따른 민권 영향 분석\nKEY Contents\nn 미국 민권위원회에 따르면 연방정부와 법 집행기관에서 얼굴인식 기술이 빠르게 도입되고\n있으나 이를 관리할 지침과 감독의 부재로 민권 문제를 초래할 위험 존재\nn 미국 민권위원회는 연방정부의 책임 있는 얼굴인식 기술 사용을 위해 운영 프로토콜 개발과\n실제 사용 상황의 얼굴인식 기술 평가 및 불평등 완화, 지역사회의 의견 수렴 등을 권고\n£연방정부의 얼굴인식 기술 도입에 대한 지침과 감독 부재로 민권 문제를 초래할 위험 존재\nn 미국 민권위원회(U.S. Commission on Civil Rights)가 2024년 9월 19일 연방정부의 얼굴인식\n기술 사용이 민권에 미치는 영향을 분석한 보고서를 발간\n∙ AI 기술의 일종인 얼굴인식 기술은 연방정부와 법 집행기관에서 빠르게 도입되고 있으며, 일례로\n법무부 연방수사국(FBI)은 범죄 수사 및 용의자 수색용 단서 확보를 위해 얼굴인식 기술을 가장 빈번히 사용\n∙ 그러나 얼굴인식 기술의 책임 있는 사용을 위한 연방 지침과 감독은 실제 활용 사례보다 뒤처졌으며,\n현재 연방정부의 얼굴인식 기술이나 여타 AI 기술 

In [10]:
for doc in docs:
    doc.metadata['filename'] = doc.metadata['source'] # source의 값을 filename이라는 키에 대입해서 새로 생성

In [ ]:
docs # filename 들어갔는지 확인

[Document(metadata={'source': './data/SPRi AI Brief_11월호_산업동향_F.pdf', 'file_path': './data/SPRi AI Brief_11월호_산업동향_F.pdf', 'page': 3, 'total_pages': 25, 'Author': 'dj', 'Creator': 'Hwp 2018 10.0.0.13947', 'Producer': 'Hancom PDF 1.3.0.547', 'CreationDate': "D:20241105150426+09'00'", 'ModDate': "D:20241105150426+09'00'", 'PDFVersion': '1.4', 'filename': './data/SPRi AI Brief_11월호_산업동향_F.pdf'}, page_content='1. 정책/법제 2. 기업/산업 3. 기술/연구 4. 인력/교육\n미국 민권위원회, 연방정부의 얼굴인식 기술 사용에 따른 민권 영향 분석\nKEY Contents\nn 미국 민권위원회에 따르면 연방정부와 법 집행기관에서 얼굴인식 기술이 빠르게 도입되고\n있으나 이를 관리할 지침과 감독의 부재로 민권 문제를 초래할 위험 존재\nn 미국 민권위원회는 연방정부의 책임 있는 얼굴인식 기술 사용을 위해 운영 프로토콜 개발과\n실제 사용 상황의 얼굴인식 기술 평가 및 불평등 완화, 지역사회의 의견 수렴 등을 권고\n£연방정부의 얼굴인식 기술 도입에 대한 지침과 감독 부재로 민권 문제를 초래할 위험 존재\nn 미국 민권위원회(U.S. Commission on Civil Rights)가 2024년 9월 19일 연방정부의 얼굴인식\n기술 사용이 민권에 미치는 영향을 분석한 보고서를 발간\n∙ AI 기술의 일종인 얼굴인식 기술은 연방정부와 법 집행기관에서 빠르게 도입되고 있으며, 일례로\n법무부 연방수사국(FBI)은 범죄 수사 및 용의자 수색용 단서 확보를 위해 얼굴인식 기술을 가장 빈번히 사용\n∙ 그러나 얼굴인식 기술의 책임 있는 사용을 위한 연방 지침과

In [12]:
# 답변의 충실성, 질문과 답변의 관련성 평가
# !pip install -q ragas==0.1.19 

In [13]:
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context, conditional
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from ragas.testset.extractor import KeyphraseExtractor
from ragas.testset.docstore import InMemoryDocumentStore
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter

c:\Users\RMARKET\anaconda3\envs\langchain\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [14]:
# 생성기
generator_llm = ChatOpenAI(model = 'gpt-4o')

In [15]:
# 비평기 (생성기와 비평기는 같은 모델을 쓰는게 좋음)
critic_llm = ChatOpenAI(model = 'gpt-4o')

In [16]:
# 임베딩 모델
embeddings = OpenAIEmbeddings(model = 'text-embedding-3-small')

In [17]:
# 텍스트 분할기 설정
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap = 100)

In [ ]:
# 구문 추출기 생성을 위한 모델 호출(RAGAS와의 호환을 위한 Wrapper)
# 키워드 중심으로 
langchain_llm = LangchainLLMWrapper(ChatOpenAI(model = 'gpt-4o'))
# 토큰수 에러나면 모델 바꿔서 사용!!!!

In [19]:
# 구문 추출기 : 문서 핵심 정보 식별 및 추출 역할
Keyphrase_extractor = KeyphraseExtractor(llm = langchain_llm)

In [20]:
# 임베딩 모델도 RAGAS와의 호환을 위해 Wrapper 적용
ragas_embeddings = LangchainEmbeddingsWrapper(embeddings)

In [21]:
# docstore 만들기
docstore = InMemoryDocumentStore(
    splitter = splitter,
    embeddings = ragas_embeddings,
    extractor = Keyphrase_extractor
)

In [22]:
# 생성기 만들기 (생성을 하면서 평가를 함)
generator = TestsetGenerator.from_langchain(
    generator_llm, # 생성기
    critic_llm, # 판별기
    ragas_embeddings, # 임베딩 모델
    docstore=docstore # 문서 저장소
)

In [23]:
# 질문 유형 분포 결정
dirstributions = {simple : 0.4, reasoning : 0.2, multi_context : 0.2, conditional : 0.2}
# simple : 간단한 질문
# reasoning : 추론을 요구하는 질문
# multi_context : 여러 맥락을 고려하는 질문
# conditional : 조건부 질문

In [24]:
testset = generator.generate_with_langchain_docs(
    documents = docs, # 문서 데이터를 전달
    test_size = 10, # 테스트 데이터 크기
    distributions=dirstributions,
    with_debugging_logs = True, # 로그 활성화
    raise_exceptions=False # 예외가 발생해도 계속 진행, 예외 발생 시 문서가 처리 되지 않거나 문서에 기록됨
)

Generating:   0%|          | 0/10 [00:00<?, ?it/s]              [ragas.testset.filters.DEBUG] context scoring: {'clarity': 1, 'depth': 2, 'structure': 2, 'relevance': 2, 'score': 1.75}
[ragas.testset.evolutions.DEBUG] keyphrases in merged node: ['Molmo-72B', 'Benchmark scores', 'GPT-4o', 'MolmoE-1B', 'Allen Institute for AI']
[ragas.testset.evolutions.INFO] seed question generated: How does GPT-4o compare to other models in terms of benchmark average scores and human preference ratings?
[ragas.testset.filters.DEBUG] context scoring: {'clarity': 1, 'depth': 2, 'structure': 2, 'relevance': 2, 'score': 1.75}
[ragas.testset.evolutions.DEBUG] keyphrases in merged node: ['AI 전문가', '기업에서의 역할', 'AI 도구 사용', 'AI의 영향', '기술 전문가']
[ragas.testset.evolutions.INFO] seed question generated: "\uae30\uc5c5\uc5d0\uc11c\uc758 \uc5ed\ud560" (Role in the company)

Question: "\uae30\uc5c5\uc5d0\uc11c AI \uc804\ubb38\uac00\ub4e4\uc758 \uc5ed\ud560\uc740 \uc5b4\ub5bb\uac8c \ud45c\ud604\ub418\ub098\uc694?"  
(Ho

In [42]:
df = testset.to_pandas()
df

,question,contexts,ground_truth,evolution_type,metadata,episode_done
0,What are the implications of the EU AI law on ...,[무결성(無缺性)이 손상될 수 있어 표준화된 데이터 수집 규약 필요\n∙ 데이터 규...,The EU AI law prohibits the use of specific ap...,simple,[{'source': './data/SPRi AI Brief_11월호_산업동향_F....,True
1,What contributions have David Baker and Demis ...,[SPRi AI Brief |\n2024-11월호\n2024년 노벨 물리학상과 화학...,David Baker and Demis Hassabis have contribute...,simple,[{'source': './data/SPRi AI Brief_11월호_산업동향_F....,True
2,AI 에이전트가 상용화되기 위해 필요한 조건은 무엇인가?,[메커니즘 대신 데이터를 우선순위에 따라 정리하고 입력에 가중치를 부여해 메모리 사...,AI 에이전트가 상용화되려면 데이터 간 연관성을 파악해 확률적으로 가장 그럴듯한 답...,simple,[{'source': './data/SPRi AI Brief_11월호_산업동향_F....,True
3,"\uae30\uc5c5\uc5d0\uc11c\uc758 \uc5ed\ud560"" (...",[때문으로 추측\nn 그러나 AI 전문가들은 기업에서 자신이 맡은 업무에 대하여 엇...,"AI 전문가들은 자신이 맡은 업무에 대하여 엇갈린 감정을 표시했으며, 자신의 업무가...",simple,[{'source': './data/SPRi AI Brief_11월호_산업동향_F....,True
4,How can investment boost AI platforms in orgs?,"[1. 정책/법제 2. 기업/산업 3. 기술/연구 4. 인력/교육\n가트너 예측, ...",Investment can boost AI platforms in organizat...,reasoning,[{'source': './data/SPRi AI Brief_11월호_산업동향_F....,True
5,How does Akita boost LLM reliability via data ...,[메커니즘 대신 데이터를 우선순위에 따라 정리하고 입력에 가중치를 부여해 메모리 사...,The answer to given question is not present in...,reasoning,[{'source': './data/SPRi AI Brief_11월호_산업동향_F....,True
6,How do gov-private partnerships balance AI ris...,"[SPRi AI Brief |\n2024-11월호\n미국 백악관 예산관리국, 정부의...",The context discusses government-private partn...,multi_context,[{'source': './data/SPRi AI Brief_11월호_산업동향_F....,True
7,How does AI safety by Google Brain affect Alph...,"[1. 정책/법제 2. 기업/산업 3. 기술/연구 4. 인력/교육\n구글 딥마인드,...",The answer to given question is not present in...,multi_context,[{'source': './data/SPRi AI Brief_11월호_산업동향_F....,True
8,How could Meta Movie Gen's 2025 Instagram inte...,"[1. 정책/법제 2. 기업/산업 3. 기술/연구 4. 인력/교육\n메타, 동영상 ...",Meta Movie Gen plans to integrate with platfor...,conditional,[{'source': './data/SPRi AI Brief_11월호_산업동향_F....,True
9,How do views on automation tech vary by countr...,[1. 정책/법제 2. 기업/산업 3. 기술/연구 4. 인력/교육\nMIT 산업성과...,The context indicates that there is a variatio...,conditional,[{'source': './data/SPRi AI Brief_11월호_산업동향_F....,True


In [ ]:
# 에러 발생으로 데이터 출력 안됐을 경우 사용!!(파일을 불러와서 확인!!)

# test_df.to_csv('data/ragas_dataset.csv', index=False)

In [44]:
# import pandas as pd
# df = pd.read_csv('data/ragas_dataset.csv')
# df.head()

In [45]:
from datasets import Dataset
test_dataset = Dataset.from_pandas(df)
test_dataset

Dataset({
    features: ['question', 'contexts', 'ground_truth', 'evolution_type', 'metadata', 'episode_done'],
    num_rows: 10
})

In [46]:
df['contexts'][0]
type(test_dataset[0]['contexts'])

list

In [ ]:
# import ast
# def convert_to_list(example):
#     contexts = ast.literal_eval(example['contexts'])
#     return{'contexts' : contexts}

In [ ]:
# test_dataset = test_dataset.map(convert_to_list)

Map: 100%|██████████| 10/10 [00:00<00:00, 668.41 examples/s]


In [ ]:
# test_dataset[0]['contexts'] # 밖의 ""가 사라짐
# type(test_dataset[0]['contexts'])

list

In [47]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

In [48]:
loader = PyMuPDFLoader('data/SPRi AI Brief_11월호_산업동향_F.pdf')
docs = loader.load()

In [49]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 50)
split_documents = text_splitter.split_documents(docs)

In [50]:
embeddings = OpenAIEmbeddings()

In [51]:
vectorstore = FAISS.from_documents(documents = split_documents, embedding = embeddings)

In [52]:
retriever = vectorstore.as_retriever()

In [53]:
prompt = PromptTemplate.from_template(
    '''너는 주어진 질문에 대답하는 AI야. 다음 검색된 context를 사용해서 question에 대답해줘.
    답을 모르면, '알 수 없습니다'라고 대답해
    
    # Context : {context}
    # Question : {question}
    # Answer : 

'''
)

In [ ]:
llm = ChatOpenAI(model = 'gpt-3.5-turbo', temperature=0) # 토큰수 부족으로 다른 모델 사용

In [72]:
chain = (
    {'context' : retriever, 'question' : RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [73]:
batch_dataset = []
for question in test_dataset['question']:
    batch_dataset.append(question)

In [74]:
batch_dataset[:3]

['What are the implications of the EU AI law on the use of AI in public spaces and high-risk AI systems?',
 'What contributions have David Baker and Demis Hassabis made to the development of new protein structures and AI models?',
 'AI 에이전트가 상용화되기 위해 필요한 조건은 무엇인가?']

In [75]:
answer = chain.batch(batch_dataset)
answer[:3]

['알 수 없습니다.',
 'David Baker and Demis Hassabis have made contributions to the development of new protein structures and AI models by developing the AlphaFold 2 AI model in 2020, which is capable of predicting protein structures for millions of proteins in just a few minutes. This has revolutionized the field of protein structure prediction and has the potential to lead to a better understanding of disease mechanisms, antibiotic resistance, and the development of new drugs and nanomaterials.',
 '알 수 없습니다.']

In [76]:
if 'answer' in test_dataset.column_names:
    test_dataset = test_dataset.remove_columns(['answer']).add_column('answer', answer)
else:
    test_dataset = test_dataset.add_column('answer',answer)

In [77]:
test_dataset

Dataset({
    features: ['question', 'contexts', 'ground_truth', 'evolution_type', 'metadata', 'episode_done', 'answer'],
    num_rows: 10
})

In [78]:
from ragas import evaluate
from ragas.metrics import answer_relevancy,faithfulness,context_recall,context_precision

In [79]:
result = evaluate(
    dataset = test_dataset,
    metrics = [
        context_recall,
        faithfulness,
        answer_relevancy,
        context_precision
    ]
)
# context_recall : 모델이 문맥에서 중요한 정보를 잘 회상, 재현 했는가
# faithfulness : 모델의 답변이 문맥에 기반해서 사실적인가(factual)
# answer_relevancy : 전체 질문에 대해 모델의 답변이 질문과 얼마나 관련성이 있는가
# context_precision : 모델이 문맥에서 필요한 정보를 정확히 활용했는가

Evaluating:  28%|██▊       | 11/40 [02:41<02:52,  5.96s/it]Exception raised in Job[38]: RateLimitError(Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4o-mini in organization org-nS8RoafnzeRRZRKTRiJUJQQC on tokens per min (TPM): Limit 200000, Used 199006, Requested 1663. Please try again in 200ms. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}})
Exception raised in Job[28]: RateLimitError(Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4o-mini in organization org-nS8RoafnzeRRZRKTRiJUJQQC on tokens per min (TPM): Limit 200000, Used 199651, Requested 2115. Please try again in 529ms. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}})
Evaluating: 100%|██████████| 40/40 [07:08<00:00, 10.72s/it]


|context_precision (문맥 정밀도)|faithfulness (신뢰성)|answer_relevancy (답변 적합성)|context_recall (문맥 재현율)|
|---|---|---|---|
|답변에 사용된 문맥이 정확했는지|답변이 문맥에 기반해 사실적(factual)인지|답변이 질문과 관련성이 있는지|답변이 문맥에서 얼마나 많은 정보를 활용했는지|
|1.0: 답변에 사용된 정보가 문맥에서 매우 정확하게 활용됨|1.0: 답변이 문맥과 매우 정확하게 일치|1.0: 답변이 질문과 매우 높은 관련성을 가짐|1.0: 답변이 문맥의 모든 중요한 정보를 잘 회상|
|0.0: 답변이 문맥과 무관하거나, 부정확한 정보를 활용|0.0: 답변이 문맥에 기반하지 않고, 사실과 다를 가능성이 큼|0.0: 답변이 질문과 관련이 없거나, 부정확|0.0: 답변이 문맥을 활용하지 못함|

In [80]:
result_df = result.to_pandas()
result_df.head()

,question,contexts,answer,ground_truth,context_recall,faithfulness,answer_relevancy,context_precision
0,What are the implications of the EU AI law on ...,[무결성(無缺性)이 손상될 수 있어 표준화된 데이터 수집 규약 필요\n∙ 데이터 규...,알 수 없습니다.,The EU AI law prohibits the use of specific ap...,NaN,NaN,NaN,NaN
1,What contributions have David Baker and Demis ...,[SPRi AI Brief |\n2024-11월호\n2024년 노벨 물리학상과 화학...,David Baker and Demis Hassabis have made contr...,David Baker and Demis Hassabis have contribute...,NaN,NaN,NaN,NaN
2,AI 에이전트가 상용화되기 위해 필요한 조건은 무엇인가?,[메커니즘 대신 데이터를 우선순위에 따라 정리하고 입력에 가중치를 부여해 메모리 사...,알 수 없습니다.,AI 에이전트가 상용화되려면 데이터 간 연관성을 파악해 확률적으로 가장 그럴듯한 답...,NaN,NaN,NaN,NaN
3,"\uae30\uc5c5\uc5d0\uc11c\uc758 \uc5ed\ud560"" (...",[때문으로 추측\nn 그러나 AI 전문가들은 기업에서 자신이 맡은 업무에 대하여 엇...,AI 전문가의 역할은 기업에서 전략적 가치를 제공하고 있다고 인식되고 있습니다. 일...,"AI 전문가들은 자신이 맡은 업무에 대하여 엇갈린 감정을 표시했으며, 자신의 업무가...",NaN,NaN,NaN,NaN
4,How can investment boost AI platforms in orgs?,"[1. 정책/법제 2. 기업/산업 3. 기술/연구 4. 인력/교육\n가트너 예측, ...",Investment can boost AI platforms in organizat...,Investment can boost AI platforms in organizat...,NaN,NaN,NaN,NaN


In [81]:
result_df.loc[:, 'context_recall':'context_precision']

,context_recall,faithfulness,answer_relevancy,context_precision
0,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN


In [ ]:
# pip install streamlit

   ---------------------------------------- 0.0/8.6 MB ? eta -:--:--
   ---------------------------------------- 8.6/8.6 MB 67.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/731.2 kB ? eta -:--:--
   --------------------------------------- 731.2/731.2 kB 29.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/6.9 MB ? eta -:--:--
   ---------------------------------------- 6.9/6.9 MB 105.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.
